In [1]:
import pandas as pd
import sys
sys.path.append('./utils')
from utils import *
from rank import *
import japanize_matplotlib
from process import * 
import tqdm

In [2]:
# データ保存先
data_folda = '../data/'

In [3]:
pseudo_time_series = pd.read_pickle(data_folda+'pseudo_time_series_with_v_retty.pkl')

* V_HOTPEPPER の変数を追加

In [4]:
pd_sf = read_table(data_folda,"V_HOTPEPPER_NAME_MASTER",columns="all",read_snowflake=False,col="*")

In [5]:
import rank
# general rank を与える
ranker=rank.Ranker()
pd_sf=ranker.add_general_rank(pd_sf)
r=pd_sf["general_rank"].value_counts().sort_index()
#S,A,B,L,F,N の順に value_countes を並べる
r=r.loc[["S","A","B","L","F","N"]]
display(r)
# cummulative sumを表示
display(pd.concat([r,r.cumsum()],axis=1))

/home/ec2-user/restaurant_scoring_ftdata/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/restaurant_scoring_ftdata/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/restaurant_scoring_ftdata/pg/./utils/rank.py:78: SettingWithCop

S    117728
A    171684
B     13856
L    229183
F        58
N      2373
Name: general_rank, dtype: int64

,general_rank,general_rank
S,117728,117728
A,171684,289412
B,13856,303268
L,229183,532451
F,58,532509
N,2373,534882


In [6]:
pd_sf.rename(columns={"general_rank":"general_rank_HOTPEPPER"},inplace=True)

In [7]:
pd_sf.columns.values

array(['acquisition_id', 'url_id', 'url', 'crawl_stamp', 'restaurant_id',
       'tabelog_name', 'tabelog_branch_name', 'tabelog_address',
       'tabelog_telephone', 'tabelog_reservation_telephone', 'name_rank',
       'name_rank_system', 'name_ratio.ratio',
       'name_ratio.master_normalized_name', 'name_ratio.normalized_name',
       'name_ratio.include', 'name_ratio.diff', 'name_only_ratio.ratio',
       'name_only_ratio.master_normalized_name',
       'name_only_ratio.normalized_name', 'name_only_ratio.include',
       'name_only_ratio.diff', 'address_rank', 'address_rank_system',
       'postal.ratio', 'postal.master_value', 'postal.value',
       'postal.diff', 'region.ratio', 'region.master_value',
       'region.value', 'region.diff', 'street_address.ratio',
       'street_address.master_value', 'street_address.value',
       'street_address.diff', 'building_name.ratio',
       'building_name.master_value', 'building_name.value',
       'building_name.diff', 'telephone_rank'

In [8]:
# info.isOfficial の空文字を0, trueを1に変換
pd_sf['infos.aggregateRating'].replace('',np.nan,inplace=True)
pd_sf['infos.aggregateRating']=pd_sf['infos.aggregateRating'].astype(float)
pd_sf['infos.ratingReview'].replace('',np.nan,inplace=True)
pd_sf['infos.ratingReview']=pd_sf['infos.ratingReview'].astype(float)
pd_sf['infos.satisfaction.percentage1'].replace('',np.nan,inplace=True)
pd_sf['infos.satisfaction.percentage1']=pd_sf['infos.satisfaction.percentage1'].astype(float)



In [9]:
pd_sf['infos.satisfaction.percentage1']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
534877   NaN
534878   NaN
534879   NaN
534880   NaN
534881   NaN
Name: infos.satisfaction.percentage1, Length: 534882, dtype: float64

In [10]:
hotpepper_list=['infos.fanCount','infos.reviewTags.count1','infos.reviewTags.count2','infos.reviewTags.count3', 'infos.reviewTags.count4',
       'infos.reviewTags.count5', 'infos.reviewTags.count6','infos.reviewTags.count7', 'infos.reviewTags.count8','infos.reviewTags.count9']
for h in hotpepper_list:
    pd_sf[h].replace('',np.nan,inplace=True)
    pd_sf[h]=pd_sf[h].astype(float)
    pd_sf[h].value_counts()

In [11]:
hotpepper_list

['infos.fanCount',
 'infos.reviewTags.count1',
 'infos.reviewTags.count2',
 'infos.reviewTags.count3',
 'infos.reviewTags.count4',
 'infos.reviewTags.count5',
 'infos.reviewTags.count6',
 'infos.reviewTags.count7',
 'infos.reviewTags.count8',
 'infos.reviewTags.count9']

In [12]:
add_list=['restaurant_id','infos.aggregateRating','infos.ratingReview','infos.satisfaction.percentage1']+hotpepper_list
          
          
          

In [13]:
pd_sf[['restaurant_id','infos.aggregateRating','infos.ratingReview','infos.satisfaction.percentage1']+hotpepper_list]

,restaurant_id,infos.aggregateRating,infos.ratingReview,infos.satisfaction.percentage1,infos.fanCount,infos.reviewTags.count1,infos.reviewTags.count2,infos.reviewTags.count3,infos.reviewTags.count4,infos.reviewTags.count5,infos.reviewTags.count6,infos.reviewTags.count7,infos.reviewTags.count8,infos.reviewTags.count9
0,d6992bb6-26c6-4176-8d77-8ea9f8154cd9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8a56c481-e1d7-4846-be05-b3a1493a8d18,NaN,NaN,NaN,66.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,be6244e8-dd52-44a2-a62a-942ae77c51b6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cad948ae-4a90-4229-9514-ca95256a890f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,a1fd98c4-012b-4bc6-8688-8dc3122b00a0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534877,7f370533-5cc2-4488-9de4-633b7beef6ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534878,47ebbfc4-6e0e-4b36-b77d-efce1f151db8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534879,bb1a332b-b58c-4481-8b2d-e3dd7921a3fd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534880,b7ec7977-6c89-4285-ab99-24db038b9238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
pseudo_time_series=pd.merge(pseudo_time_series,pd_sf[['general_rank_HOTPEPPER','restaurant_id','infos.aggregateRating','infos.ratingReview','infos.satisfaction.percentage1']+hotpepper_list],on="restaurant_id",how="left")

In [ ]:
pseudo_time_series.to_pickle(data_folda+'pseudo_time_series_with_v_hotpepper.pkl')